# Sacrapping Data Comment from Youtube

In [2]:
!pip install selenium --quiet


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Scrap

In [2]:
data = []

with Chrome(executable_path=r'C:\webdrivers\chromedriver.exe') as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(
        "https://www.youtube.com/watch?v=rxzhWUWV9yM&ab_channel=KOMPASTV")

    for item in range(10):
        wait.until(EC.visibility_of_element_located(
            (By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)


C:\Users\Dio\AppData\Local\Temp/ipykernel_31424/3059915953.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\webdrivers\chromedriver.exe') as driver:


In [3]:
import pandas as pd
df = pd.DataFrame(data, columns=['comment'])
df

,comment
0,ID\nSkip navigation\nSign in\nIni Jagoan Pilpr...
1,
2,Jika masarakat Indonesia ingin ketentraman dan...
3,Kita doakan agar datang pemimpin nasional yg b...
4,"Semoga kita jgn salah pilih, karna menyangkut ..."
...,...
624,
625,
626,
627,


### Preporcessing and Cleaning

In [4]:
import re
import string
import numpy as np

# Text Cleaning


def cleaning(text):
    # HTML Tag Removal

    text = re.compile(
        '<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [5]:
from nltk.stem import PorterStemmer
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang,
                emoji_to_words, remove_html])
# steamming
ps = PorterStemmer()

In [6]:
df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)

In [7]:
print(df.isna().sum())

comment            0
comment (clean)    9
dtype: int64


In [8]:
df.dropna(inplace=True)

### Data Clean

In [9]:
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nIni Jagoan Pilpr...,id skip navigation sign ini ini jagoan pilpres...
2,Jika masarakat Indonesia ingin ketentraman dan...,jika masarakat indonesia ingin ketentraman dan...
3,Kita doakan agar datang pemimpin nasional yg b...,kita doakan agar datang pemimpin nasional yang...
4,"Semoga kita jgn salah pilih, karna menyangkut ...",semoga kita jangan salah pilih karena menyangk...
5,Ganjar menang di survei Anis menang di hati ma...,ganjar meg di survei anis meg di hati masyarakat
...,...,...
617,Kalo pilpres tidak pake sistim degital tetap k...,kalo pilpres tidak pakai sistim degital tetap ...
618,Sapa aja boleh Asal jgn ganjar dan puan,sapa saja boleh asal jangan ganjar dan puan
619,3 Mestro Pemimpin Milineal Bangsa Mengubah wuj...,mestro pemimpin milineal bangsa mengubah wuju...
620,"Kalo begitu, PDIp lh yg mau trs berkuasa",kalo begitu pdip boleh yang mau terus berkuasa


### Export to CSV

In [10]:
# to csv
df.to_csv('datacomment.csv', index=False)